# Training a CNN classifier with VAE augmented data

### Imports

In [1]:
import mlflow
import torch
import vae
from utils.mlflow import backend_stores
from utils.trainer import TrainingArguments
from vae.generation import augment_dataset_using_per_class_vaes, augment_dataset_using_single_vae, augmentations
from vae.models import VAEConfig
from evaluation.models import CNNMNIST
from evaluation.train_utils import train_model
from utils.data import load_datasets

vae.models.base.model_store = "pretrained_models/MNIST"

### Seeding, loading data & setting up mlflow logging

In [2]:
SEED = 1337
DATASET = "MNIST"
DATASET_LIMIT = 50
# set the backend store uri of mlflow
mlflow.set_tracking_uri(getattr(backend_stores, DATASET))
# seed torch
torch.manual_seed(SEED)
# load datasets
train_dataset, vae_train_dataset, val_dataset, test_dataset = load_datasets(DATASET)

### The parameters for the classification task

In [3]:
training_args = TrainingArguments(
    total_steps=5000,
    batch_size=32,
    validation_intervall=200,
    save_model=False,
    seed=SEED,
    early_stopping=False,
    early_stopping_window=20,
    save_best_metric="best_acc",
)

### Parameters for data augmentation

In [4]:
MULTI_VAE = False
VAE_EPOCHS = 100
Z_DIM = 10
BETA = 1.0
AUGMENTATION = None
augmentation_params = {"k": 9}

In [5]:
#from utils.data import create_train_dataset
#create_train_dataset("MNIST", 50, 1338)

## Training the CNN

In [6]:
# create a vae config
vae_config = VAEConfig(z_dim=Z_DIM, beta=BETA)
# start mlflow run in experiment
mlflow.set_experiment(f"CNN Z_DIM {Z_DIM}" if AUGMENTATION else "CNN Baseline")
print("Training CNN" + f" augmented with {Z_DIM=}, {BETA=}, {VAE_EPOCHS=}" if AUGMENTATION is not None else "")
with mlflow.start_run(run_name=AUGMENTATION or "baseline"):
    # log dataset limit
    mlflow.log_param("dataset_limit", DATASET_LIMIT)
    # perform data augmentation if specified
    if AUGMENTATION is not None:
        # log MULTI_VAE flag
        mlflow.log_param("multi_vae", MULTI_VAE)
        # log vae epochs
        mlflow.log_param("vae_epochs", VAE_EPOCHS)
        # if MULTI_VAE, augment data of each label seperately
        if MULTI_VAE:
            train_dataset = augment_dataset_using_per_class_vaes(
                train_dataset, vae_config, VAE_EPOCHS, AUGMENTATION, augmentation_params, seed=SEED
            )
        # else: conventional vae data augmentation
        else:
            train_dataset = augment_dataset_using_single_vae(
                train_dataset, vae_config, VAE_EPOCHS, AUGMENTATION, augmentation_params, seed=SEED
            )
    # train cnn
    results = train_model(
        model=CNNMNIST(),
        training_args=training_args,
        train_dataset=train_dataset,
        dev_dataset=val_dataset,
        test_dataset=test_dataset,
        seed=SEED,
    )
    # print the results
    print(results)

INFO: 'CNN Baseline' does not exist. Creating a new experiment



Training epoch 2500: 100%|██████████| 5000/5000 [08:49<00:00,  9.43it/s, loss=0.05]

Evaluating model.


{'eval_loss': 1.2057771896362304, 'eval_acc': 0.7537, 'eval_f1': 0.7536013372046073, 'eval_best_acc': 0.7537}
